# Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd 'drive/My Drive/ERA V1/Assignment 9/src'

Mounted at /content/drive
/content/drive/My Drive/ERA V1/Assignment 9/src


In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchsummary import summary
from tqdm import tqdm
from model import *
from utils import *
from dataset import cifar10Dataset, get_loader
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [3]:
SEED = 1

# CUDA?
use_cuda = torch.cuda.is_available()
print("CUDA Available?", use_cuda)

# For reproducibility
torch.manual_seed(SEED)

if use_cuda:
    torch.cuda.manual_seed(SEED)

CUDA Available? False


In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transforms = A.Compose([
        A.HorizontalFlip(p=0.05),
        A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=10),
        A.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)), #normalize
        A.CoarseDropout(max_holes = 1, max_height=16, max_width=16, min_holes = 1, min_height=16, min_width=16, fill_value=0.49, mask_fill_value =None),
        ToTensorV2()])

test_transforms = A.Compose([
        A.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)), #normalize
        ToTensorV2()])

In [5]:
train_data = cifar10Dataset('./data', train=True, download=True, transform=train_transforms) #
test_data = cifar10Dataset('./data', train=False, download=True, transform=test_transforms) #

Files already downloaded and verified
Files already downloaded and verified


In [6]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Dataloader Arguments & Test/Train Dataloaders


In [7]:
# train dataloader and test loader
batch_size = 128
train_loader, test_loader = get_loader(train_data, test_data, batch_size, use_cuda)

# Model Params



In [8]:
from model import Net

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
           Dropout-3           [-1, 32, 32, 32]               0
              ReLU-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           9,216
       BatchNorm2d-6           [-1, 32, 32, 32]              64
           Dropout-7           [-1, 32, 32, 32]               0
              ReLU-8           [-1, 32, 32, 32]               0
            Conv2d-9           [-1, 32, 15, 15]           9,216
      BatchNorm2d-10           [-1, 32, 15, 15]              64
          Dropout-11           [-1, 32, 15, 15]               0
             ReLU-12           [-1, 32, 15, 15]               0
           Conv2d-13           [-1, 64, 15, 15]          18,432
      BatchNorm2d-14           [-1, 64,

# Training and Testing

In [ ]:
train_losses_t = []
test_losses_t = []
train_acc_t = []
test_acc_t = []

model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 80
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train_losses, train_acc = train(model, device, train_loader, optimizer, epoch)
    train_losses_t += train_losses
    train_acc_t.append(train_acc)
    test_losses, test_acc = test(model, device, test_loader)
    test_losses_t += test_losses
    test_acc_t.append(test_acc)

EPOCH: 0


Loss=1.519556999206543 Batch_id=781 Accuracy=37.60: 100%|██████████| 782/782 [05:52<00:00,  2.22it/s]



Test set: Average loss: 1.5407, Accuracy: 4682/10000 (46.82%)

EPOCH: 1


Loss=1.3034554719924927 Batch_id=57 Accuracy=48.25:   7%|▋         | 58/782 [01:08<23:05,  1.91s/it]

In [ ]:
plot_loss_acc(train_losses_t, train_acc_t, test_losses_t, test_acc_t)